<div style='text-align: center; margin-bottom: 20px;'>
    <h1>🛰️ SeekerLight</h1>
</div>

In [ ]:
# Parameters and imports (this cell will be hidden)
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from datetime import datetime, timedelta
import os
import warnings
import shutil
import time
import math
import traceback
from functools import partial
from io import StringIO
from contextlib import redirect_stdout

# Suppress warnings
warnings.filterwarnings('ignore')

# Initialize Earth Engine
try:
    # Redirect stdout to suppress survey message
    temp_stdout = StringIO()
    with redirect_stdout(temp_stdout):
        ee.Initialize(project='ee-sergiyk1974')
    print("✓ Earth Engine initialized successfully")
except Exception as e:
    print("Authenticating with Earth Engine...")
    # Redirect stdout to suppress survey message
    temp_stdout = StringIO()
    with redirect_stdout(temp_stdout):
        ee.Authenticate()
        ee.Initialize()
    print("✓ Earth Engine authenticated and initialized")

# Initialize Sentinel-1 collection
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')

# File to store AOIs
AOI_MASTER_FILE = 'custom_aois_master.py'

# Copy AOIs from sat-scanner if our file doesn't exist
if not os.path.exists(AOI_MASTER_FILE):
    src_file = '../sat-scanner/custom_aois_master.py'
    if os.path.exists(src_file):
        shutil.copy2(src_file, AOI_MASTER_FILE)

# Function to read existing AOIs
def read_existing_aois():
    if not os.path.exists(AOI_MASTER_FILE):
        return {}
    try:
        namespace = {'ee': ee}
        with open(AOI_MASTER_FILE) as f:
            exec(f.read(), namespace)
        return namespace.get('CUSTOM_AOIS', {})
    except Exception as e:
        print(f"Error reading AOIs: {str(e)}")
        return {}

# Function to write AOIs to file
def write_aois_file(aois_dict):
    content = [
        'import ee\n',
        '# Master file for custom AOIs\n',
        'CUSTOM_AOIS = {\n'
    ]
    
    for name, aoi_info in sorted(aois_dict.items()):
        coords_str = str(aoi_info['coords']).replace('], [', '],\n            [')
        content.extend([
            f"    '{name}': {{\n",
            f"        'coords': {coords_str},\n",
            f"        'description': '{aoi_info.get('description', '')}',\n",
            f"        'area_km2': {aoi_info.get('area_km2', 0):.2f}\n",
            "    },\n"
        ])
    
    content.extend([
        "}\n\n",
        "# Create ee.Geometry objects for each AOI\n",
        "for name, aoi_info in CUSTOM_AOIS.items():\n",
        "    aoi_info['geometry'] = ee.Geometry.Polygon([aoi_info['coords']])\n"
    ])
    
    with open(AOI_MASTER_FILE, 'w') as f:
        f.writelines(content)

# Initialize AOIs
aois = read_existing_aois()
if aois:
    print("✓ Areas of Interest loaded successfully")
else:
    print("✓ Ready to create new Areas of Interest")

In [ ]:
# Create the application interface
title = HTML('<div style="text-align:center; margin: 20px 0;"><h2>Satellite Data Viewer</h2></div>')
display(title)

# Create map
m = geemap.Map(
    center=[47.2357, 39.7015],
    zoom=8,
    height='600px',
    layout=widgets.Layout(width='800px', height='600px')
)

# Add basemap
m.add_basemap('HYBRID')

# Add drawing controls
m.add_draw_control()

# Create widgets
bounds_text = widgets.HTML(
    value='Current Bounds: Not set',
    layout=widgets.Layout(width='300px')
)

aoi_selector = widgets.Dropdown(
    options=[('Select an AOI', None)] + [(name, name) for name in sorted(aois.keys())],
    value=None,
    description='Area of Interest:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Create date range widgets with default filters for last 30 days
default_end_date = datetime.now()
default_start_date = default_end_date - timedelta(days=30)

start_date = widgets.DatePicker(
    description='Start Date:',
    value=default_start_date,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

end_date = widgets.DatePicker(
    description='End Date:',
    value=default_end_date,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

# Quick date filter selection
date_filter = widgets.Dropdown(
    options=[
        ('Last 30 days', 30),
        ('Last 60 days', 60),
        ('Last 90 days', 90),
        ('Last 180 days', 180),
        ('Last 365 days', 365),
        ('Custom', 0)
    ],
    value=30,
    description='Quick Filter:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

def on_date_filter_change(change):
    if change['new'] == 0:  # Custom selection
        return
    
    new_end_date = datetime.now()
    new_start_date = new_end_date - timedelta(days=change['new'])
    
    start_date.value = new_start_date
    end_date.value = new_end_date

date_filter.observe(on_date_filter_change, names='value')

update_button = widgets.Button(
    description='Update View',
    button_style='primary',
    icon='refresh',
    layout=widgets.Layout(width='150px')
)

save_aoi_button = widgets.Button(
    description='Save Current AOI',
    button_style='success',
    icon='save',
    layout=widgets.Layout(width='150px')
)

delete_aoi_button = widgets.Button(
    description='Delete AOI',
    button_style='danger',
    icon='trash',
    layout=widgets.Layout(width='150px')
)

export_button = widgets.Button(
    description='Export Data',
    button_style='info',
    icon='download',
    layout=widgets.Layout(width='150px')
)

output = widgets.Output(
    layout=widgets.Layout(width='300px', height='100px')
)

# Layout widgets
controls = widgets.VBox([
    widgets.HTML('<h3>Controls</h3>'),
    bounds_text,
    widgets.HBox([
        aoi_selector,
        delete_aoi_button
    ], layout=widgets.Layout(justify_content='space-between')),
    widgets.HBox([
        widgets.VBox([
            widgets.HTML('<h4>Date Range</h4>'),
            date_filter,
            start_date,
            end_date
        ])
    ], layout=widgets.Layout(justify_content='space-between')),
    widgets.HBox([update_button, save_aoi_button, export_button], 
                layout=widgets.Layout(justify_content='space-between', margin='10px 0')),
    output
], layout=widgets.Layout(width='350px', margin='0 0 0 20px'))

# Create main layout
main_layout = widgets.HBox([
    m,
    controls
], layout=widgets.Layout(justify_content='space-between'))

# Display layout
display(main_layout)

# Event handlers
def get_map_bounds():
    """Get the current map bounds as an ee.Geometry"""
    bounds = m.bounds
    coords = [[
        [bounds[0][1], bounds[0][0]],  # SW
        [bounds[0][1], bounds[1][0]],  # SE
        [bounds[1][1], bounds[1][0]],  # NE
        [bounds[1][1], bounds[0][0]],  # NW
        [bounds[0][1], bounds[0][0]]   # SW (close polygon)
    ]]
    return ee.Geometry.Polygon(coords)

def on_update_click(b):
    with output:
        clear_output()
        try:
            # Get geometry based on selection or current bounds
            if aoi_selector.value:
                geometry = aois[aoi_selector.value]['geometry']
                bounds_text.value = f'Using AOI: {aoi_selector.value}'
            else:
                geometry = get_map_bounds()
                bounds_text.value = f'Using viewport bounds: {m.bounds}'
            
            # Convert dates to ee.Date objects
            ee_start_date = ee.Date(start_date.value.strftime('%Y-%m-%d'))
            ee_end_date = ee.Date(end_date.value.strftime('%Y-%m-%d'))
            
            # Filter collection and sort by date
            collection = (sentinel1
                .filterDate(ee_start_date, ee_end_date)
                .filterBounds(geometry)
                .sort('system:time_start'))
            
            # Get the size of the collection
            size = collection.size().getInfo()
            print(f"Found {size} images in view")
            
            if size < 2:
                print("Need at least 2 images for temporal comparison")
                return
            
            # Create list of images
            image_list = collection.toList(size)
            
            # Create three temporal versions
            first_image = ee.Image(image_list.get(0))
            middle_image = ee.Image(image_list.get(size // 2))
            last_image = ee.Image(image_list.get(size - 1))
            
            # Create RGB composite using temporal difference
            composite = ee.Image.cat([
                first_image.select('VH'),
                middle_image.select('VH'),
                last_image.select('VH')
            ]).clip(geometry)
            
            print("Processing view update...")
            
            # Try to update map layers with retry mechanism
            if update_map_layers(m, composite, geometry):
                print(f"Updated view with temporal comparison")
                print(f"Date Range: {start_date.value.strftime('%Y-%m-%d')} to {end_date.value.strftime('%Y-%m-%d')}")
            else:
                print("Failed to update view after multiple attempts.")
                print("Please try clicking update again.")
        except Exception as e:
            print(f"Error updating view: {str(e)}")
            print("Traceback:")
            traceback.print_exc()

def update_map_layers(m, composite, geometry, retries=3, delay=1):
    """Helper function to update map layers with retry mechanism"""
    for attempt in range(retries):
        try:
            # Clear all layers except the base layer
            if len(m.layers) > 1:
                m.layers = m.layers[:1]  # Keep only the first (base) layer
            
            time.sleep(delay)  # Small delay to let server catch up
            
            # Add basemap if needed
            if len(m.layers) == 0:
                m.add_basemap('HYBRID')
            
            # Add the composite layer with specified visualization
            vis_params = {
                'bands': ['VH', 'VH_1', 'VH_2'],
                'min': -25,
                'max': 0,
                'gamma': 1.4
            }
            
            # Create the tile layer first
            sar_layer = geemap.ee_tile_layer(
                ee_object=composite,
                vis_params=vis_params,
                name='SAR Temporal Composite'
            )
            
            # Add the SAR layer
            m.add_layer(sar_layer)
            
            time.sleep(delay)  # Small delay between layer additions
            
            # Add boundary outline
            style = {
                'color': 'yellow' if not aoi_selector.value else 'red',
                'fillOpacity': 0.0,
                'weight': 2
            }
            
            # Create and add the boundary layer
            bounds_layer = geemap.ee_tile_layer(
                ee_object=geometry,
                vis_params=style,
                name='Boundary'
            )
            m.add_layer(bounds_layer)
            
            # Center map if using AOI
            if aoi_selector.value:
                m.center_object(geometry, zoom=12)
            
            return True
        except Exception as e:
            print(f"Attempt {attempt + 1} failed:")
            traceback.print_exc()
            if attempt < retries - 1:  # Don't sleep on the last attempt
                time.sleep(delay)
            continue
    return False

def on_save_aoi(b):
    with output:
        clear_output()
        try:
            # Get drawn features
            drawn_features = m.draw_features
            if not drawn_features:
                print("Please draw an AOI first using the drawing tools")
                return
                
            # Get the last drawn feature
            feature = drawn_features[-1]
            if feature['geometry']['type'] != 'Polygon':
                print("Please draw a polygon")
                return
                
            # Get coordinates
            coords = feature['geometry']['coordinates'][0]
            
            # Calculate area
            geometry = ee.Geometry.Polygon([coords])
            area_km2 = geometry.area().getInfo() / 1_000_000
            
            # Create AOI name
            base_name = "Custom AOI"
            name = base_name
            counter = 1
            while name in aois:
                name = f"{base_name} {counter}"
                counter += 1
            
            # Add to AOIs
            aois[name] = {
                'coords': coords,
                'description': 'Created from SeekerLight',
                'area_km2': area_km2,
                'geometry': geometry
            }
            
            # Save to file
            write_aois_file(aois)
            
            # Update dropdown
            aoi_selector.options = [('Select an AOI', None)] + [(name, name) for name in sorted(aois.keys())]
            aoi_selector.value = name
            
            print(f"AOI saved as '{name}' (Area: {area_km2:.2f} km²)")
            
        except Exception as e:
            print(f"Error saving AOI: {str(e)}")
            traceback.print_exc()

def delete_aoi(b):
    with output:
        clear_output()
        if not aoi_selector.value:
            print("Please select an AOI to delete")
            return
        
        try:
            name_to_delete = aoi_selector.value
            
            # Remove from dictionary
            aois.pop(name_to_delete)
            
            # Update the file
            write_aois_file(aois)
            
            # Update dropdown options
            aoi_selector.options = [('Select an AOI', None)] + [(name, name) for name in sorted(aois.keys())]
            aoi_selector.value = None
            
            print(f"AOI '{name_to_delete}' deleted successfully")
            
            # Clear the map
            m.clear_layers()
            m.add_basemap('HYBRID')
            
        except Exception as e:
            print(f"Error deleting AOI: {str(e)}")

def export_data(b):
    with output:
        clear_output()
        if not aoi_selector.value:
            print("Please select an AOI for export")
            return
        
        try:
            # Get the selected AOI
            selected_aoi = aois[aoi_selector.value]['geometry']
            
            # Get the current composite
            ee_start_date = ee.Date(start_date.value.strftime('%Y-%m-%d'))
            ee_end_date = ee.Date(end_date.value.strftime('%Y-%m-%d'))
            
            # Filter collection and sort by date
            collection = (sentinel1
                .filterDate(ee_start_date, ee_end_date)
                .filterBounds(selected_aoi)
                .sort('system:time_start'))
            
            # Get the size of the collection
            size = collection.size().getInfo()
            
            if size < 2:
                print("Need at least 2 images for temporal comparison")
                return
            
            # Create list of images
            image_list = collection.toList(size)
            
            # Create three temporal versions
            first_image = ee.Image(image_list.get(0))
            middle_image = ee.Image(image_list.get(size // 2))
            last_image = ee.Image(image_list.get(size - 1))
            
            # Create RGB composite using temporal difference
            composite = ee.Image.cat([
                first_image.select('VH'),
                middle_image.select('VH'),
                last_image.select('VH')
            ]).clip(selected_aoi)
            
            # Calculate bounds for export
            bounds = selected_aoi.bounds()
            
            # Get coordinates for dimensions calculation
            coords = bounds.coordinates().getInfo()[0]
            
            # Calculate rough dimensions in meters
            x_min, y_min = coords[0]
            x_max, y_max = coords[2]
            
            # Convert degrees to approximate meters (rough estimation)
            meters_per_degree = 111319.9  # approximate meters per degree at equator
            width_meters = abs(x_max - x_min) * meters_per_degree * math.cos(math.radians((y_max + y_min) / 2))
            height_meters = abs(y_max - y_min) * meters_per_degree
            
            # Calculate dimensions that are multiples of 256
            target_resolution = 10  # 10 meters per pixel
            width_pixels = math.ceil(width_meters / target_resolution / 256) * 256
            height_pixels = math.ceil(height_meters / target_resolution / 256) * 256
            
            # Start export task
            task = ee.batch.Export.image.toDrive(
                image=composite,
                description=f'SAR_{start_date.value.strftime("%Y%m%d")}',
                folder='SeekerLight_Exports',
                fileNamePrefix=f'SAR_{start_date.value.strftime("%Y%m%d")}',
                region=bounds,
                dimensions=[width_pixels, height_pixels],
                crs='EPSG:4326',
                maxPixels=1e13,
                formatOptions={
                    'cloudOptimized': True
                }
            )
            
            task.start()
            print(f"Export started!")
            print(f"Date Range: {start_date.value.strftime('%Y-%m-%d')} to {end_date.value.strftime('%Y-%m-%d')}")
            print(f"Target Resolution: ~{target_resolution} meters")
            print(f"Dimensions: {width_pixels}x{height_pixels} pixels")
            print(f"Check your Google Drive's 'SeekerLight_Exports' folder.")
            print(f"Note: Export may take some time to complete.")
            
        except Exception as e:
            print(f"Error during export: {str(e)}")
            print("Traceback:")
            traceback.print_exc()

# Connect event handlers
update_button.on_click(on_update_click)
save_aoi_button.on_click(on_save_aoi)
export_button.on_click(export_data)
delete_aoi_button.on_click(delete_aoi)

# Initial update
bounds_text.value = f'Current Bounds: {m.bounds}'